In [25]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_timetable"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8020,application_1589299642358_2543,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7965,application_1589299642358_2485,pyspark,idle,Link,Link,
7971,application_1589299642358_2491,pyspark,idle,Link,Link,
7978,application_1589299642358_2498,pyspark,idle,Link,Link,
7984,application_1589299642358_2505,pyspark,busy,Link,Link,
7985,application_1589299642358_2506,pyspark,idle,Link,Link,
7987,application_1589299642358_2508,pyspark,busy,Link,Link,
7992,application_1589299642358_2514,pyspark,idle,Link,Link,
7994,application_1589299642358_2517,pyspark,idle,Link,Link,
7998,application_1589299642358_2521,pyspark,busy,Link,Link,


In [26]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7933,application_1589299642358_2451,pyspark,idle,Link,Link,
7965,application_1589299642358_2485,pyspark,idle,Link,Link,
7971,application_1589299642358_2491,pyspark,idle,Link,Link,
7978,application_1589299642358_2498,pyspark,idle,Link,Link,
7984,application_1589299642358_2505,pyspark,busy,Link,Link,
7985,application_1589299642358_2506,pyspark,idle,Link,Link,
7987,application_1589299642358_2508,pyspark,busy,Link,Link,
7992,application_1589299642358_2514,pyspark,idle,Link,Link,
7994,application_1589299642358_2517,pyspark,idle,Link,Link,
7998,application_1589299642358_2521,pyspark,busy,Link,Link,


In [28]:
import pyspark.sql.functions as f

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
trips = spark.read.orc("/data/sbb/timetables/orc/trips/")
calendar = spark.read.orc("/data/sbb/timetables/orc/calendar/")
routes = spark.read.orc("/data/sbb/timetables/orc/routes/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
trips.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 1 row

In [10]:
calendar.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 1 row

In [11]:
routes.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|               |       Bus|       700|
+-----------+---------+----------------+---------------+----------+----------+
only showing top 1 row

In [12]:
# select all services that are scheduled on mondays
monday_service_id = (
    calendar
    .where(calendar.monday == True)
    .select('service_id')
    .dropDuplicates()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# keep only trips that are scheduled on mondays
monday_trips = (
    trips
    .join(f.broadcast(monday_service_id), 'service_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# 

@f.udf
def make_id()

In [21]:
# add route information to trips
route_trips = (
    trips
    .join(routes, 'route_id')
    .withColumn('ist_trip_id', f.concat_ws(':', f.lit('85'), 'agency_id', 'trip_short_name'))
    .withColumn('ist_line_id', f.concat_ws(':', f.lit('85'), 'agency_id', 'route_short_name'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
route_trips.dropDuplicates(['route_type'])

An error was encountered:
Invalid status code '404' from http://iccluster044.iccluster.epfl.ch:8998/sessions/7975 with error payload: "Session '7975' not found."


In [22]:
route_trips.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------------------+------------------+---------------+------------+---------+----------------+---------------+----------+----------+----------------+----------------+
|route_id   |service_id|trip_id              |trip_headsign     |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc|route_type|train_identifier|other_identifier|
+-----------+----------+---------------------+------------------+---------------+------------+---------+----------------+---------------+----------+----------+----------------+----------------+
|1-1-C-j19-1|TA+b0001  |5.TA.1-1-C-j19-1.3.R |Zofingen, Altachen|108            |1           |723      |1               |               |Bus       |700       |85:723:108      |85:723:1        |
|1-1-C-j19-1|TA+b0001  |7.TA.1-1-C-j19-1.3.R |Zofingen, Altachen|112            |1           |723      |1               |               |Bus       |700       |85:723:112      |85:723:1        |
|1-1-C-j19-1|TA+b0001  |9.TA.1

In [23]:
route_trips.dropDuplicates(['route_desc']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+------------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+----------------+----------------+
|route_id      |service_id|trip_id                 |trip_headsign            |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc   |route_type|train_identifier|other_identifier|
+--------------+----------+------------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+----------------+----------------+
|57-3-Y-j19-1  |TA+b0030  |9.TA.57-3-Y-j19-1.9.H   |Zürich HB                |9211           |0           |11       |3-Y             |               |TGV          |102       |85:11:9211      |85:11:3-Y       |
|40-12-Y-j19-1 |TA+b000n  |14.TA.40-12-Y-j19-1.7.H |Como S. Giovanni         |315            |0           |11       |12-Y            |               |Eurocity  

In [ ]:
@f.udf
def trip_id(id_):
    parts = id_.split(':')
    if len(parts) > 1:
        return parts[1]
    else:
        return None

In [25]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/").repartition(10, 'stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
stop_time.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_sequence: short (nullable = true)
 |-- pickup_type: byte (nullable = true)
 |-- drop_off_type: byte (nullable = true)

In [27]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:35:00|      04:35:00|8500313:0:2|            6|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:52:00|      04:52:00|8500304:0:2|           11|          0|            0|
|25.TA.1-1-B-j19-1...|    05:58:00|      05:58:00|8500021:0:5|            3|          0|            0|
|25.TA.1-1-B-j19-1...|    06:05:00|      06:05:00|8500313:0:2|            6|          0|            0|
|25.TA.1-1-B-j19-1...|    06:22:00|      06:22:00|8500304:0:2|           11|          0|            0|
|28.TA.1-1-B-j19-1...|    06:28:00|      06:28:00|8500021:0:5|           

In [28]:
@f.udf
def parent_station_id(stop_id):
    return stop_id.split(':')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .withColumn('stop_id', parent_station_id(stop_time.stop_id))
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|25.TA.1-1-B-j19-1...|    05:58:00|      05:58:00|8500021|            3|
|25.TA.1-1-B-j19-1...|    06:05:00|      06:05:00|8500313|            6|
|25.TA.1-1-B-j19-1...|    06:22:00|      06:22:00|8500304|           11|
|28.TA.1-1-B-j19-1...|    06:28:00|      06:28:00|8500021|            3|
|28.TA.1-1-B-j19-1...|    06:35:00|      06:35:00|8500313|            6|
|28.TA.1-1-B-j19-1...|    06:52:00|      06:52:00|8500304|           11|
|45.TA.1-1-B-j19-1...|    09:28:00|      09:28:00|8500021|            3|
|45.TA.1-1-B-j19-1...|    09:35:00|      09:35:00|8500313|            6|
|45.TA.1-1-B-j19-1...|    09:52:00|      09:52:00|8500304|           11|
|53.TA.1-1-B-j19-1...|    10:28:00|      10:28:00|8500021|            3|
+--------------------+------------+--------------+-

In [31]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
stop_ids = [row['stop_id'] for row in stations.collect()]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
stop_ids = spark.sparkContext.broadcast(stop_ids)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
filtered = stop_time.where(stop_time.stop_id.isin(stop_ids.value))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
filtered.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|1.TA.1-11-B-j19-1...|    06:16:00|      06:16:00|8594304|            3|
|2.TA.1-11-B-j19-1...|    06:46:00|      06:46:00|8594304|            3|
|3.TA.1-11-B-j19-1...|    07:16:00|      07:16:00|8594304|            3|
|4.TA.1-11-B-j19-1...|    07:46:00|      07:46:00|8594304|            3|
|5.TA.1-11-B-j19-1...|    08:16:00|      08:16:00|8594304|            3|
|6.TA.1-11-B-j19-1...|    08:46:00|      08:46:00|8594304|            3|
|7.TA.1-11-B-j19-1...|    09:16:00|      09:16:00|8594304|            3|
|8.TA.1-11-B-j19-1...|    09:46:00|      09:46:00|8594304|            3|
|9.TA.1-11-B-j19-1...|    10:16:00|      10:16:00|8594304|            3|
|10.TA.1-11-B-j19-...|    10:46:00|      10:46:00|8594304|            3|
+--------------------+------------+--------------+-

In [41]:
timetable = (
    filtered
    .orderBy(['trip_id', 'stop_sequence'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
timetable.write.format("csv").save("/user/gflueck/timetable_zurich.csv", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
(
    stop_time
    .groupBy(['trip_id', 'arrival_time'])
    .agg(
        f.collect_list(f.col('stop_sequence')).alias('sequence'), 
        f.collect_list(f.col('stop_id')).alias('stops'), 
        f.countDistinct(f.col('stop_id')).alias('count'),
    )
    .where(f.col('count') > 5)
).show(10, False)

In [14]:
departure = stop_time.alias('departure')
arrival = stop_time.withColumn('stop_sequence', stop_time.stop_sequence - 1).alias('arrival')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
timetable = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
    .select(
        departure.trip_id,
        departure.stop_sequence,
        departure.stop_id.alias('start_id'),
        departure.departure_time.alias('start_time'),
        arrival.arrival_time.alias('stop_time'),
        arrival.stop_id.alias('stop_id')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
timetable.write.format("csv").save("/user/gflueck/timetable.csv", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…